In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import layers

In [2]:
mnist_dataset, mnist_info= tfds.load(name='mnist', with_info= True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits ['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale (image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

2022-01-18 11:56:04.239271: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-18 11:56:04.240340: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
Buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(Buffer_size)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

Batch_size = 100

train_data = train_data.batch(Batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))

2022-01-18 11:56:04.445782: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-18 11:56:05.844307: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [4]:

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation= 'relu', input_shape = (28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation= 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation= 'relu')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))

In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
num_epochs = 5
model.fit(train_data, epochs = num_epochs, validation_data=(validation_inputs,validation_targets), verbose=2)

Epoch 1/5
540/540 - 39s - loss: 0.2361 - accuracy: 0.9276 - val_loss: 0.0644 - val_accuracy: 0.9807
Epoch 2/5
540/540 - 29s - loss: 0.0580 - accuracy: 0.9820 - val_loss: 0.0499 - val_accuracy: 0.9832
Epoch 3/5
540/540 - 28s - loss: 0.0399 - accuracy: 0.9873 - val_loss: 0.0387 - val_accuracy: 0.9880
Epoch 4/5
540/540 - 30s - loss: 0.0312 - accuracy: 0.9901 - val_loss: 0.0406 - val_accuracy: 0.9872
Epoch 5/5
540/540 - 29s - loss: 0.0251 - accuracy: 0.9918 - val_loss: 0.0296 - val_accuracy: 0.9918


In [8]:
test_loss,test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}, Test accuracy: {1:.2f}%'.format)

1/1 [==============================] - 2s 2s/step - loss: 0.0311 - accuracy: 0.9912
<built-in method format of str object at 0x7f8a1a1ee2d0>
